In [49]:
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import redis
import json

In [16]:
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [53]:
redis_host = 'localhost'
redis_port = 6379
redis_client = redis.Redis(host=redis_host, port=redis_port, db=2)

In [ ]:
keys = redis_client.keys("nytsentences:*")

In [57]:
batch_size=64
index_min,index_max=700*batch_size,700*batch_size+batch_size
results = []
#62336 error
for i in range(700,len(keys)//batch_size):
    print(f"{i*64}/{len(keys)}")
    dates = [str(key).split(":")[1] for key in keys[index_min:index_max]]
    batch = [redis_client.get(key).decode('utf-8') for key in keys[index_min:index_max]]
    result=model(tokenizer.batch_encode_plus(batch,padding=True,truncation=True,return_tensors='tf'))
    for res in list(zip(result[0].numpy().tolist(),dates)):
        results.append(res)
    index_max+=batch_size
    index_min+=batch_size
len(results)

44800/88284
44864/88284
44928/88284
44992/88284
45056/88284
45120/88284
45184/88284
45248/88284
45312/88284
45376/88284
45440/88284
45504/88284
45568/88284
45632/88284
45696/88284
45760/88284
45824/88284
45888/88284
45952/88284
46016/88284
46080/88284
46144/88284
46208/88284
46272/88284
46336/88284
46400/88284
46464/88284
46528/88284
46592/88284
46656/88284
46720/88284
46784/88284
46848/88284
46912/88284
46976/88284
47040/88284
47104/88284
47168/88284
47232/88284
47296/88284
47360/88284
47424/88284
47488/88284
47552/88284
47616/88284
47680/88284
47744/88284
47808/88284
47872/88284
47936/88284
48000/88284
48064/88284
48128/88284
48192/88284
48256/88284
48320/88284
48384/88284
48448/88284
48512/88284
48576/88284
48640/88284
48704/88284
48768/88284
48832/88284
48896/88284
48960/88284
49024/88284
49088/88284
49152/88284
49216/88284
49280/88284
49344/88284
49408/88284
49472/88284
49536/88284
49600/88284
49664/88284
49728/88284
49792/88284
49856/88284
49920/88284
49984/88284
50048/88284
5011

KeyboardInterrupt: 

In [58]:
with open("../flat_files/nyt_polarities_3.json","w") as f:
    f.write(json.dumps(results))

In [56]:
len(results)

44736